In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms,models
from torch.utils.data import Subset
from torchsummary import summary
import numpy as np

In [6]:
batch_size = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

数据：Fashion-MNIST 3x299x299，仅取 n %

In [ ]:
# 构建数据预处理流水线：按顺序执行多个数据变换操作
transform = transforms.Compose([
    # 将图像缩放到299×299尺寸（InceptionV3要求的输入尺寸），默认保持宽高比，短边缩至299后长边自适应，最终通过中心裁剪/拉伸为299×299
    transforms.Resize(299),
    # 将FashionMNIST的单通道灰度图转换为3通道灰度图（InceptionV3要求3通道输入，三通道值相同）
    transforms.Grayscale(num_output_channels=3),
    # 将PIL图像/NumPy数组转换为PyTorch张量，同时将像素值从[0,255]归一化到[0.0,1.0]
    transforms.ToTensor()
])

# 加载FashionMNIST完整训练集，自动下载至'data'目录，应用上述预处理流水线
train_full = datasets.FashionMNIST('data',train=True,download=True,transform=transform)

# 加载FashionMNIST完整测试集，自动下载至'data'目录，应用相同的预处理流水线保证数据格式一致
test_full = datasets.FashionMNIST('data',train=False,download=True,transform=transform)


100%|██████████| 26421880/26421880 [00:08<00:00, 2983013.32it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 87679.45it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:02<00:00, 2197041.29it/s]


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [22]:
n = 10 #根据实际选取不同比例数据
# 1. 创建一个固定随机种子的numpy随机数生成器（RNG）
# seed=42 保证每次运行代码生成的随机数结果完全一致（可复现性）
# default_rng是numpy 1.17+推荐的新随机数生成接口，比np.random更规范
rng = np.random.default_rng(42)  

# 2. 从训练集全量数据的索引中随机抽取子集索引
# len(train_full)：获取训练集全量数据的样本总数（比如FashionMNIST训练集是60000）
# len(train_full)//n：计算要抽取的样本数（//是整数除法，比如n=10时，抽取60000//10=6000个）
# replace=False：不重复抽样（保证每个索引只选一次，避免同一个样本被多次选中）
# 最终train_idx是一个一维数组，存放被选中的训练集样本索引
train_idx = rng.choice(len(train_full), len(train_full)//n, replace=False)

# 3. 同理，从测试集全量数据的索引中随机抽取子集索引
# len(test_full)：测试集全量样本数（比如FashionMNIST测试集是10000）
# len(test_full)//n：测试集要抽取的样本数（比如n=10时抽取1000个）
# 最终test_idx是测试集被选中样本的索引数组
test_idx = rng.choice(len(test_full), len(test_full)//n, replace=False)

In [23]:
train_idx,test_idx

(array([ 5296, 40657, 52882, ..., 39272, 46436,  5355], dtype=int64),
 array([9329, 9609, 3000, 3425, 5966, 1625, 5802, 1673, 5075, 7181, 3204,
        5098, 2815, 4801,  683, 2636, 6114, 8952, 2470, 8069, 7643,  754,
        7056, 3072, 6687,  253, 3095, 1634, 1933, 1002, 2609, 8991, 1308,
        4404, 6707,  911, 2991,  863,  277, 3313, 4051, 3602,  314, 8345,
        4372, 5500, 7818, 8471,   45, 3687, 4151,  202, 6501, 8634, 1606,
        7785, 6919, 9240, 3586, 5430,  498, 7504, 2174, 8800, 4098, 1491,
        1329, 8437, 2699, 3978, 5904, 3286, 8771,  621,   69, 8772,  563,
         489, 3084, 9082, 8677, 3742, 4599, 8972, 7667, 9203,  809,  716,
          42, 6718, 2928, 3809, 7166, 8547, 2288, 4250, 6424, 1509,  305,
        3406, 1290, 9937, 1984, 1770, 5642, 5108, 4357, 1709,  897, 9832,
         715, 6109, 8195, 8542, 7892, 5444, 5482, 7255,  628, 7476, 5365,
        9634, 4713, 3755, 4669, 1641, 3100, 5831, 8565, 8335, 9747, 5850,
        5255, 1715, 6976, 6949, 8153, 7731

In [ ]:
# PyTorch核心数据加载器：将数据集封装为可迭代的批次迭代器
train_loader = torch.utils.data.DataLoader(Subset(train_full,train_idx),# 从完整数据集中截取指定子集
                                           batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(Subset(test_full,test_idx),
                                           batch_size=batch_size,shuffle=True)

导入官方Inception-V3和权重，只换 fc

In [25]:
model = models.inception_v3(weights=models.Inception_V3_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features,10)
model = model.to(device)
optimizer = optim.Adam(model.parameters(),lr=1e-4)

summary(model,input_size=(3,299,299))

训练/测试

In [26]:
epochs = 10
accs,losses =[],[]
for epoch in range(epochs):
    model.train()
    for x,y in train_loader:
        x,y = x.to(device),y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        out = outputs.logits  # 主分类输出
        loss = F.cross_entropy(out,y)
        loss.backward()
        optimizer.step()

    model.eval()
    correct,total_loss = 0,0.
    with torch.no_grad():
        for x,y in test_loader:
            x,y = x.to(device),y.to(device)
            out = model(x)
            total_loss +=F.cross_entropy(out,y).item()
            correct +=(out.argmax(1)==y).sum().item()
        
    acc = correct / len(test_loader.dataset)
    avg_loss = total_loss / len(test_loader)
    accs.append(acc)
    losses.append(avg_loss)
    print(f'epoch{epoch}:loss={avg_loss:.4f},acc={acc:.4f}')

epoch0:loss=0.3711,acc=0.8680
epoch1:loss=0.3311,acc=0.8960
epoch2:loss=0.2609,acc=0.9160
epoch3:loss=0.2851,acc=0.9030
epoch4:loss=0.2565,acc=0.9200
epoch5:loss=0.3223,acc=0.9060
epoch6:loss=0.2958,acc=0.9200
epoch7:loss=0.3218,acc=0.9130
epoch8:loss=0.3321,acc=0.9160
epoch9:loss=0.4530,acc=0.8990


In [27]:
transform

Compose(
    Resize(size=299, interpolation=bilinear, max_size=None, antialias=True)
    Grayscale(num_output_channels=3)
    ToTensor()
)

In [28]:
train_full

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=299, interpolation=bilinear, max_size=None, antialias=True)
               Grayscale(num_output_channels=3)
               ToTensor()
           )

In [29]:
test_full

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: Compose(
               Resize(size=299, interpolation=bilinear, max_size=None, antialias=True)
               Grayscale(num_output_channels=3)
               ToTensor()
           )

In [30]:
train_loader

In [31]:
test_loader

In [32]:
model

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [33]:
model.fc

Linear(in_features=2048, out_features=10, bias=True)